# Statistics

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

def analyze_filter_conditions(input_file, output_dir):
    # 读取SQANTI输出文件
    df = pd.read_csv(input_file, sep='\t')

    # 定义过滤条件
    conditions = {
        'basic_filter': df['RTS_stage'] == False,
        'canonical': df['all_canonical'] == 'canonical',
        'coverage_filter': df['min_cov'] >= 5,
        'detection': df['sample_no'] >= 3,
        'predicted_NMD': df['predicted_NMD'] == False,
        'three_prime_filter': (abs(df['diff_to_TTS']) <= 100) | (df['within_polyA_site'] == True)
    }

    # 创建输出目录
    output_dir.mkdir(parents=True, exist_ok=True)

    # 遍历每个条件，计算满足和未满足的数量，生成饼图
    for condition_name, condition_series in conditions.items():
        satisfied = condition_series.sum()
        not_satisfied = len(df) - satisfied

        # 打印统计信息
        print(f"{condition_name}:")
        print(f"  Satisfied: {satisfied}")
        print(f"  Not Satisfied: {not_satisfied}")
        print(f"  Percentage Satisfied: {satisfied / len(df) * 100:.2f}%\n")

        # 绘制饼图
        labels = [f'Satisfied ({satisfied})', f'Not Satisfied ({not_satisfied})']
        sizes = [satisfied, not_satisfied]
        colors = ['#4CAF50', '#FFC107']
        plt.figure(figsize=(6, 6))
        plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
        plt.title(f"Condition: {condition_name}")

        # 保存饼图
        output_path = output_dir / f"{condition_name}_pie_chart.png"
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()

    print(f"All pie charts have been saved in: {output_dir}")

# 使用示例
from pathlib import Path
analyze_filter_conditions(
    input_file='./020.brca_classifcation_sampeCount_loc_symbol_meanPSI_DR_deltaPSI_mane_drug_target_drug_FC_org_func_singleton.tsv',
    output_dir=Path('./QC_pie_charts/')
)


/tmp/ipykernel_3171741/1754681303.py:6: DtypeWarning: Columns (1,2,5,6,7,14,15,16,19,24,28,29,36,38,40,42,43,45,46,50,51,52,53,54,55,57,58,63,65,66,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file, sep='\t')


basic_filter:
  Satisfied: 124282
  Not Satisfied: 20759
  Percentage Satisfied: 85.69%

canonical:
  Satisfied: 99446
  Not Satisfied: 45595
  Percentage Satisfied: 68.56%

coverage_filter:
  Satisfied: 91258
  Not Satisfied: 53783
  Percentage Satisfied: 62.92%

detection:
  Satisfied: 51993
  Not Satisfied: 93048
  Percentage Satisfied: 35.85%

predicted_NMD:
  Satisfied: 95278
  Not Satisfied: 49763
  Percentage Satisfied: 65.69%

three_prime_filter:
  Satisfied: 101562
  Not Satisfied: 43479
  Percentage Satisfied: 70.02%

All pie charts have been saved in: QC_pie_charts


## filtering with stanard criteria

In [1]:
import pandas as pd
import numpy as np

def filter_sqanti_results(input_file, output_file):
    # 读取SQANTI输出文件
    df = pd.read_csv(input_file, sep='\t')

    # 分离isoform_id以AF-开头的行，处理NaN值
    af_rows = df[df['isoform_id'].fillna('').str.startswith('AF-')]
    
    # 打印过滤前每个structural_category的数量，排除AF-开头的行
    print("Original structural_category counts (excluding AF-isoforms):")
    print(df[~df['isoform_id'].fillna('').str.startswith('AF-')]['structural_category'].value_counts())

    # 定义过滤函数
    def apply_filters(row):
        # 基本过滤条件：RTS_stage必须为False
        basic_filter = (row['RTS_stage'] == False)
        
        # 判断是否为canonical splice sites
        canonical = (row['all_canonical'] == 'canonical')
        
        # 读取覆盖度过滤：min_cov必须大于等于5
        coverage_filter = (row['min_cov'] >= 5)
        
        # detection rate
        detection = (row['sample_no'] >= 3)
        
        # NMD detection
        predicted_NMD = row['predicted_NMD'] == False
        
        # 3'末端可靠性过滤：满足3个条件之一
        three_prime_filter = (
            (abs(row['diff_to_TTS']) <= 100) or
            (row['within_polyA_site'] == True)
        )
        
        # 根据structural_category应用不同的过滤条件
        if row['structural_category'] == 'full-splice_match':
            # 对于 full-splice_match 不进行过滤，直接保留
            return True
        elif row['structural_category'] in ['novel_not_in_catalog']:
            return basic_filter
        elif row['structural_category'] in ['novel_in_catalog']:
            return basic_filter
        elif row['structural_category'] in ['incomplete-splice_match']:
            return basic_filter and three_prime_filter and detection and canonical and coverage_filter
        else:  # 其他类别 (genic, antisense, fusion, intergenic)
            return basic_filter and three_prime_filter and detection and canonical and coverage_filter
    
    # 对除isoform_id以AF-开头的行之外的所有行应用过滤条件
    df_filtered = df[~df['isoform_id'].fillna('').str.startswith('AF-')].copy()
    df_filtered = df_filtered[df_filtered.apply(apply_filters, axis=1)]

    # 合并保留的AF-开头的行和过滤后的其他行
    final_df = pd.concat([af_rows, df_filtered])

    # 保存过滤后的结果
    final_df.to_csv(output_file, sep='\t', index=False)

    # 打印过滤后的每个structural_category的数量，排除AF-开头的行
    print("\nFiltered structural_category counts (excluding AF-isoforms):")
    print(final_df[~final_df['isoform_id'].fillna('').str.startswith('AF-')]['structural_category'].value_counts())

    # 打印结果统计
    print(f"\nOriginal isoforms (excluding AF-isoforms): {len(df[~df['isoform_id'].fillna('').str.startswith('AF-')])}")
    print(f"Filtered isoforms (excluding AF-isoforms): {len(final_df[~final_df['isoform_id'].fillna('').str.startswith('AF-')])}")
    print(f"Percentage retained (excluding AF-isoforms): {len(final_df[~final_df['isoform_id'].fillna('').str.startswith('AF-')]) / len(df[~df['isoform_id'].fillna('').str.startswith('AF-')]) * 100:.2f}%")

# 使用示例
filter_sqanti_results('./020.brca_classifcation_sampeCount_loc_symbol_meanPSI_DR_deltaPSI_mane_drug_target_drug_FC_org_func_singleton.tsv', './filtering_transcriptome/021.filtered_0916_gff3_classification_merged_detection_rate.tsv')


/tmp/ipykernel_300181/3046263358.py:6: DtypeWarning: Columns (1,2,5,6,7,14,15,16,19,24,28,29,36,38,40,42,43,45,46,50,51,52,53,54,55,57,58,63,65,66,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file, sep='\t')


Original structural_category counts (excluding AF-isoforms):
structural_category
novel_not_in_catalog       32456
incomplete-splice_match    31494
novel_in_catalog           27272
full-splice_match          24176
intergenic                 10089
fusion                      2252
antisense                   1507
genic                         88
genic_intron                   1
Name: count, dtype: int64

Filtered structural_category counts (excluding AF-isoforms):
structural_category
novel_not_in_catalog       30467
novel_in_catalog           26697
full-splice_match          24176
incomplete-splice_match     9524
fusion                        53
antisense                      7
genic                          3
intergenic                     2
Name: count, dtype: int64

Original isoforms (excluding AF-isoforms): 129337
Filtered isoforms (excluding AF-isoforms): 90929
Percentage retained (excluding AF-isoforms): 70.30%


## filter with adding NMD filtering condition

In [ ]:
import pandas as pd
import numpy as np

def filter_sqanti_results(input_file, output_file):
    # 读取SQANTI输出文件
    df = pd.read_csv(input_file, sep='\t')

    # 分离isoform_id以AF-开头的行，处理NaN值
    af_rows = df[df['isoform_id'].fillna('').str.startswith('AF-')]
    
    # 打印过滤前每个structural_category的数量，排除AF-开头的行
    print("Original structural_category counts (excluding AF-isoforms):")
    print(df[~df['isoform_id'].fillna('').str.startswith('AF-')]['structural_category'].value_counts())

    # 定义过滤函数
    def apply_filters(row):
        # 基本过滤条件：RTS_stage必须为False
        basic_filter = (row['RTS_stage'] == False)
        
        # 判断是否为canonical splice sites
        canonical = (row['all_canonical'] == 'canonical')
        
        # 读取覆盖度过滤：min_cov必须大于等于5
        coverage_filter = (row['min_cov'] >= 5)
        
        # detection rate
        detection = (row['sample_no'] >= 3)
        
        # NMD detection
        predicted_NMD = row['predicted_NMD'] != True
        
        # 3'末端可靠性过滤：满足3个条件之一
        three_prime_filter = (
            (abs(row['diff_to_TTS']) <= 100) or
            (row['within_polyA_site'] == True)
        )
        
        # 根据structural_category应用不同的过滤条件
        if row['structural_category'] == 'full-splice_match':
            # 对于 full-splice_match 不进行过滤，直接保留
            return True
        elif row['structural_category'] in ['novel_not_in_catalog']:
            return basic_filter and predicted_NMD
        elif row['structural_category'] in ['novel_in_catalog']:
            return basic_filter and predicted_NMD
        elif row['structural_category'] in ['incomplete-splice_match']:
            return basic_filter and three_prime_filter and detection and canonical and coverage_filter and predicted_NMD
        else:  # 其他类别 (genic, antisense, fusion, intergenic)
            return basic_filter and three_prime_filter and detection and canonical and coverage_filter and predicted_NMD
    
    # 对除isoform_id以AF-开头的行之外的所有行应用过滤条件
    df_filtered = df[~df['isoform_id'].fillna('').str.startswith('AF-')].copy()
    df_filtered = df_filtered[df_filtered.apply(apply_filters, axis=1)]

    # 合并保留的AF-开头的行和过滤后的其他行
    final_df = pd.concat([af_rows, df_filtered])

    # 保存过滤后的结果
    final_df.to_csv(output_file, sep='\t', index=False)

    # 打印过滤后的每个structural_category的数量，排除AF-开头的行
    print("\nFiltered structural_category counts (excluding AF-isoforms):")
    print(final_df[~final_df['isoform_id'].fillna('').str.startswith('AF-')]['structural_category'].value_counts())

    # 打印结果统计
    print(f"\nOriginal isoforms (excluding AF-isoforms): {len(df[~df['isoform_id'].fillna('').str.startswith('AF-')])}")
    print(f"Filtered isoforms (excluding AF-isoforms): {len(final_df[~final_df['isoform_id'].fillna('').str.startswith('AF-')])}")
    print(f"Percentage retained (excluding AF-isoforms): {len(final_df[~final_df['isoform_id'].fillna('').str.startswith('AF-')]) / len(df[~df['isoform_id'].fillna('').str.startswith('AF-')]) * 100:.2f}%")

# 使用示例
filter_sqanti_results('./020.brca_classifcation_sampeCount_loc_symbol_meanPSI_DR_deltaPSI_mane_drug_target_drug_FC_org_func_singleton.tsv', './filtering_transcriptome/021.brca_classifcation_QC_sampeCount_loc_symbol_meanPSI_DR_deltaPSI_mane_drug_target_drug_FC_org_func_singleton_noNMD.tsv')


/tmp/ipykernel_3171741/2786381726.py:6: DtypeWarning: Columns (1,2,5,6,7,14,15,16,19,24,28,29,36,38,40,42,43,45,46,50,51,52,53,54,55,57,58,63,65,66,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file, sep='\t')


Original structural_category counts (excluding AF-isoforms):
structural_category
novel_not_in_catalog       32456
incomplete-splice_match    31494
novel_in_catalog           27272
full-splice_match          24176
intergenic                 10089
fusion                      2252
antisense                   1507
genic                         88
genic_intron                   1
Name: count, dtype: int64

Filtered structural_category counts (excluding AF-isoforms):
structural_category
full-splice_match          24176
novel_not_in_catalog       21676
novel_in_catalog           19419
incomplete-splice_match     9374
fusion                        38
antisense                      7
genic                          2
intergenic                     1
Name: count, dtype: int64

Original isoforms (excluding AF-isoforms): 129337
Filtered isoforms (excluding AF-isoforms): 74693
Percentage retained (excluding AF-isoforms): 57.75%
